### YouTube API is used to gather statistics on Trending videos and associated channels.

In [2]:
!pip install google-api-python-client

   ---------------------------------------- 0.0/12.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/12.1 MB ? eta -:--:--
    --------------------------------------- 0.3/12.1 MB ? eta -:--:--
   -- ------------------------------------- 0.8/12.1 MB 1.4 MB/s eta 0:00:09
   --- ------------------------------------ 1.0/12.1 MB 1.4 MB/s eta 0:00:08
   ---- ----------------------------------- 1.3/12.1 MB 1.5 MB/s eta 0:00:08
   ----- ---------------------------------- 1.6/12.1 MB 1.5 MB/s eta 0:00:08
   ------ --------------------------------- 2.1/12.1 MB 1.5 MB/s eta 0:00:07
   ------- -------------------------------- 2.4/12.1 MB 1.6 MB/s eta 0:00:07
   -------- ------------------------------- 2.6/12.1 MB 1.5 MB/s eta 0:00:07
   ---------- ----------------------------- 3.1/12.1 MB 1.6 MB/s eta 0:00:06
   ----------- ---------------------------- 3.4/12.1 MB 1.6 MB/s eta 0:00:06
   ------------ --------------------------- 3.7/12.1 MB 1.6 MB/s eta 0:00:06
   ------------- ---

In [28]:
from googleapiclient.discovery import build
import googleapiclient.errors

In [40]:
import requests
import pandas as pd

In [30]:
service_name = "youtube"
version = "v3"
api_key = "api_key"

In [31]:
youtube = build(service_name, version, developerKey=api_key)

**Function to obtain the trending videos in India**

In [47]:
def YT_api_request(youtube):
    all_results =[]
    next_page_token = None

    while True:
        request = youtube.videos().list(
        part="snippet,contentDetails,status,statistics,topicDetails",
        chart="mostPopular",
        maxResults=50,
        regionCode="IN",
        pageToken = next_page_token #to fetch all the pages
        )
    
        response = request.execute()
    
        # Add the current page's results to the all_results list
        all_results.extend(response.get("items", []))

        # Check if there is a next page
        next_page_token = response.get("nextPageToken")
        if not next_page_token:
            break  # No more pages, exit the loop

    return all_results
        


In [35]:
video_api_result = YT_api_request(youtube)

In [39]:
#converting api response into a dataframe
if isinstance(video_api_result, list):
  data =[]
  for item in video_api_result:
      if isinstance(item, dict):
          video_data = {
            'video_id': item.get('id'),
            'publishedAt': item['snippet'].get('publishedAt'),
            'channelId': item['snippet'].get('channelId'),
            'title': item['snippet'].get('title'),
            'thumbnails': item['snippet'].get('thumbnails', {}),
            'channelTitle': item['snippet'].get('channelTitle'),
            'tags': item['snippet'].get('tags', []),
            'defaultAudioLanguage': item['snippet'].get('defaultAudioLanguage'),
            'duration': item['contentDetails'].get('duration'),
            'dimension': item['contentDetails'].get('dimension'),
            'definition': item['contentDetails'].get('definition'),
            'caption': item['contentDetails'].get('caption'),
            'licensedContent': item['contentDetails'].get('licensedContent'),
            'contentRating': item['contentDetails'].get('contentRating'),
            'projection': item['contentDetails'].get('projection'),
            'madeForKids': item['status'].get('madeForKids'),
            'viewCount': item['statistics'].get('viewCount'),
            'likeCount': item['statistics'].get('likeCount'),
            'favoriteCount': item['statistics'].get('favoriteCount'),
            'commentCount': item['statistics'].get('commentCount'),
            'topicDetails': item['topicDetails'].get('topicCategories')
          }
          data.append(video_data)

In [43]:
df = pd.DataFrame(data)
df.shape

(160, 21)

In [45]:
df.to_csv("D:\Project_DA\Youtube Api\TrendingVideo_YTdata.csv")

In [51]:
channel_id_list = df['channelId'].tolist()


In [102]:
len(channel_id_list)

160

**Function to get Channel statistics of the trending videos**

In [115]:
def YT_api_request_channel(youtube, channel_id_list):
    channel_id_list = [cid for cid in channel_id_list if cid]
    max_ids_per_request = 50
    all_responses = []

    for i in range(0, len(channel_id_list), max_ids_per_request):
        batch = channel_id_list[i:i + max_ids_per_request]
        next_page_token = None
        
        while True:
            request = youtube.channels().list(
                part="contentDetails,snippet,statistics,topicDetails,status,brandingSettings",
                id=','.join(batch),
                pageToken=next_page_token
            )
            response = request.execute()
            all_responses.extend(response['items'])
            
            next_page_token = response.get('nextPageToken')
            if not next_page_token:
                break
    
    return all_responses


In [116]:
channel_response = YT_api_request_channel(youtube, channel_id_list)

In [129]:
#parsing the api response into a dataframe
data_Ch = []

for channel in channel_response:
    snippet = channel['snippet']
    status = channel['status']
    statistics = channel['statistics']
    brandingSettings = channel['brandingSettings']

    data_Ch.append({
        'channelId': channel['id'],
        'channel_publishedAt': snippet['publishedAt'],
        'channelTitle': snippet['title'],
        'channel_viewCount': statistics['viewCount'],
        'subscriberCount': statistics['subscriberCount'],
        'hiddenSubscriberCount': statistics['hiddenSubscriberCount'],
        'videoCount': statistics['videoCount'],
        'keywords': brandingSettings['channel'].get('keywords', []),
        'defaultLang': brandingSettings['channel'].get('defaultLanguage')
    })




In [130]:
df_2 = pd.DataFrame(data_Ch)
df_2.shape

(137, 9)

In [140]:
df_2.to_csv('D:\Project_DA\Youtube Api\ChannelInfo_TrendingVideo.csv')

**Why my dataframe has only 137 rows whereas I have requested for 160 channel ids???
 Because some trending videos may have same channel ids**

In [132]:
df_channels = set(df['channelId'])
list_channels = set(channel_id_list)

In [133]:
missing_channels = list_channels-df_channels

In [134]:
missing_channels

set()